# ExtVid

## Import

In [ ]:
import cv2
import numpy as np
import moviepy.editor as mp
import os

## Define ExtVid

In [ ]:
def extvid(path_input_video, path_output_video):
    
    input_video = cv2.VideoCapture(path_input_video)
    
    if not input_video.isOpened():
        print("Input video can't be opened")
        return
    
    WIDTH = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    HEIGHT = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    FPS = input_video.get(cv2.CAP_PROP_FPS)
    
    HALF_WIDTH = int(WIDTH/2)
    HALF_HEIGHT = int(HEIGHT/2)
    
    QUARTER_WIDTH = int(HALF_WIDTH/2)
    QUARTER_HEIGHT = int(HALF_HEIGHT/2)
    
    PATH_MASK_IMG = './img/mask.jpg'
    PATH_WHITE_IMG = './img/white.jpg'

    # encoder(for mp4)
    FOURCC = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    
    # output file name, encoder, fps, size(fit to image size)
    output_video = cv2.VideoWriter(path_output_video, FOURCC, FPS, (WIDTH, HEIGHT))
    
    if not output_video.isOpened():
        print("Output video can't be opened")
        return

    # For internal masked image
    mask_img = cv2.imread(PATH_MASK_IMG)
    mask_img = cv2.resize(mask_img, (WIDTH, HEIGHT), interpolation=cv2.INTER_NEAREST)
    
    white_img = cv2.imread(PATH_WHITE_IMG)
    white_img_half = cv2.resize(white_img, (HALF_WIDTH, HALF_HEIGHT), interpolation=cv2.INTER_NEAREST)
    
    # For external masked image
    white_img_vcon = cv2.resize(white_img, (HALF_WIDTH, QUARTER_HEIGHT), interpolation=cv2.INTER_NEAREST)
    white_img_hcon = cv2.resize(white_img, (QUARTER_WIDTH, HEIGHT), interpolation=cv2.INTER_NEAREST)
    
    
    while True:
        ret, frame = input_video.read()
        if ret:
            half_frame = cv2.resize(frame, dsize=(HALF_WIDTH, HALF_HEIGHT), interpolation=cv2.INTER_AREA)
            
            external_masked_img = cv2.vconcat([white_img_vcon, half_frame, white_img_vcon])
            external_masked_img = cv2.hconcat([white_img_hcon, external_masked_img, white_img_hcon])
            
            internal_masked_img = cv2.bitwise_or(frame, mask_img)
            
            blended_img = cv2.bitwise_and(internal_masked_img, external_masked_img)
        
            blurred_img = cv2.blur(blended_img, (51,51))
        
            result_img = cv2.seamlessClone(half_frame, blurred_img, white_img_half, (HALF_WIDTH, HALF_HEIGHT), cv2.NORMAL_CLONE)
            
            output_video.write(result_img)
        else:
            break
            
    output_video.release()

    # add audio
    clip_input = mp.VideoFileClip(path_input_video).subclip()
    clip_output = mp.VideoFileClip(path_output_video).subclip()
    clip_output = clip_output.set_audio(clip_input.audio)
    os.remove(path_output_video)
    clip_output.write_videofile(path_output_video, temp_audiofile="temp-audio.m4a", remove_temp=True, codec="libx264", audio_codec="aac")

## Use ExtVid as below
You can use ExtVid only to prepare input video file.

### ExtVid　Arguments

1. path of input video

2. path of output video

In [ ]:
extvid('./input.mp4', './output.mp4')